In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


In [4]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index


# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [12]:
character_to_ix = {}
for i in range(26):
    character_to_ix[chr(ord('a')+i)] = i
print(character_to_ix)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [5]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [6]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        # print(tag_scores, targets)
        
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])
tensor([[-0.0462, -4.0106, -3.6096],
        [-4.8205, -0.0286, -3.9045],
        [-3.7876, -4.1355, -0.0394],
        [-0.0185, -4.7874, -4.6013],
        [-5.7881, -0.0186, -4.1778]])


In [47]:
def character_process_data(seq, tags, tag_to_idx):
    global character_to_ix
    input_list = []
    target_list = []
    for word, tag in zip(seq, tags):
        embed_input = [character_to_ix[i] for i in word.lower()]
        embed_target = [tag_to_ix[tag] for _ in range(len(embed_input))]
        input_list.append(embed_input)
        target_list.append(embed_target)
    return input_list, target_list


In [51]:
model2 = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(character_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    input_list, target_list = character_process_data(training_data[0][0], training_data[0][1], tag_to_ix)
    # print(input_list)
    # print(target_list)
    # inputs = prepare_sequence(training_data[0][0], word_to_ix)
    for inp in input_list:
        tag_scores = model2(torch.tensor(inp))
        print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model2.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        input_list, target_list = character_process_data(training_data[0][0], training_data[0][1], tag_to_ix)

        for inp, target in zip(input_list, target_list):
            tag_scores = model2(torch.tensor(inp))
            # print(torch.tensor(inp))
            # print(torch.tensor(target))
            loss = loss_function(tag_scores, torch.tensor(target))
            loss.backward()

        optimizer.step()

tensor([[-1.3427, -0.8317, -1.1921],
        [-1.3340, -0.7787, -1.2816],
        [-1.3858, -0.8243, -1.1669]])
tensor([[-1.3221, -0.9552, -1.0536],
        [-1.2236, -0.8827, -1.2304],
        [-1.3707, -0.9384, -1.0361]])
tensor([[-1.4397, -0.8951, -1.0373],
        [-1.4099, -0.8532, -1.1093],
        [-1.4215, -0.8685, -1.0815]])
tensor([[-1.3427, -0.8317, -1.1921],
        [-1.3340, -0.7787, -1.2816],
        [-1.3858, -0.8243, -1.1669]])
tensor([[-1.4397, -0.8951, -1.0373],
        [-1.3763, -0.8679, -1.1157],
        [-1.3289, -0.7994, -1.2530],
        [-1.3699, -0.7810, -1.2451],
        [-1.3579, -0.8126, -1.2070]])


In [52]:
with torch.no_grad():
    input_list, target_list = character_process_data(training_data[0][0], training_data[0][1], tag_to_ix)
    # print(input_list)
    # print(target_list)
    # inputs = prepare_sequence(training_data[0][0], word_to_ix)
    for inp in input_list:
        tag_scores = model2(torch.tensor(inp))
        print(tag_scores)

tensor([[-5.0838e-03, -5.9908e+00, -5.9641e+00],
        [-8.6151e-04, -7.5420e+00, -8.0143e+00],
        [-7.5776e-04, -7.4850e+00, -8.5369e+00]])
tensor([[-7.0276e+00, -3.2161e-03, -6.0645e+00],
        [-7.4925e+00, -1.9047e-03, -6.6109e+00],
        [-1.0315e+01, -6.6783e-04, -7.3627e+00]])
tensor([[-6.4776e+00, -9.8602e-01, -4.6936e-01],
        [-5.5515e+00, -5.5390e+00, -7.8427e-03],
        [-5.9457e+00, -5.8968e+00, -5.3797e-03]])
tensor([[-5.0838e-03, -5.9908e+00, -5.9641e+00],
        [-8.6151e-04, -7.5420e+00, -8.0143e+00],
        [-7.5776e-04, -7.4850e+00, -8.5369e+00]])
tensor([[-6.4776e+00, -9.8602e-01, -4.6936e-01],
        [-7.9744e+00, -5.8629e-03, -5.2027e+00],
        [-7.6790e+00, -1.2270e-03, -7.1772e+00],
        [-8.0469e+00, -7.8349e-04, -7.6776e+00],
        [-9.8114e+00, -7.3513e-04, -7.2934e+00]])


In [53]:
with torch.no_grad():
    input_list, target_list = character_process_data(['The', 'boy', 'plays', 'the', 'ball'], ["DET", "NN", "V", "DET", "NN"], tag_to_ix)
    # print(input_list)
    # print(target_list)
    # inputs = prepare_sequence(training_data[0][0], word_to_ix)
    for inp in input_list:
        tag_scores = model2(torch.tensor(inp))
        print(tag_scores)

tensor([[-5.0838e-03, -5.9908e+00, -5.9641e+00],
        [-8.6151e-04, -7.5420e+00, -8.0143e+00],
        [-7.5776e-04, -7.4850e+00, -8.5369e+00]])
tensor([[-0.0361, -4.1057, -3.9661],
        [-0.0402, -3.3201, -5.7278],
        [-0.4549, -1.0099, -6.7037]])
tensor([[-2.7923e+00, -6.4537e-02, -6.7114e+00],
        [-4.5291e+00, -1.1228e-02, -7.8887e+00],
        [-6.9707e+00, -1.8566e-02, -4.0481e+00],
        [-7.5370e+00, -3.6140e-03, -5.7846e+00],
        [-7.3521e+00, -1.1166e-03, -7.6526e+00]])
tensor([[-5.0838e-03, -5.9908e+00, -5.9641e+00],
        [-8.6151e-04, -7.5420e+00, -8.0143e+00],
        [-7.5776e-04, -7.4850e+00, -8.5369e+00]])
tensor([[-3.6068e-02, -4.1057e+00, -3.9661e+00],
        [-4.2641e+00, -1.8467e+00, -1.8853e-01],
        [-6.2282e+00, -4.3152e-02, -3.2124e+00],
        [-8.3039e+00, -2.1538e-03, -6.2638e+00]])
